In [ ]:
import torch
import pytorch_gb as pgb
from example_modules import GPT2

In [ ]:
model2 = GPT2(nlayers=4,dropout=0)
context1 = torch.tensor([[ 464, 5440, 4534]])
d = {"src":context1}

In [ ]:
import warnings ; warnings.filterwarnings("ignore")
GPT2_ag = pgb.make_all_graphs(model2,d)

In [ ]:
pgb.print_all_graphs(GPT2_ag,"GPT2",False)

In [ ]:
tf_mod = torch.nn.Transformer(nhead=16, num_encoder_layers=1)
src = torch.rand((10, 32, 512))
tgt = torch.rand((10, 32, 512))
tf_inputs = {"src":src,"tgt":tgt}

In [ ]:
tf_ag = pgb.make_all_graphs(tf_mod,tf_inputs)

In [ ]:
# pgb.print_all_graphs(tf_ag,"nn.tf",False) -> the K_graph is to complicated for graphviz to handle it

===================
## -> rockmate
===================

In [ ]:
import rockmate as rk

In [ ]:
rk.test_everything(model2,d,show_debug=True)